<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/dev.Alejandro/API_Connection_20250731.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API Connection to ABQ Transit Database
- Database: abq-transit-test


## Add "YOUR_API_KEY" to your secrets
- Secret name = YOUR_API_KEY
  - For API Key email: Alejandrovasquez.ddds@gmail.com

In [ ]:
from google.colab import userdata


In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime
import plotly.express as px
import folium

# Your API endpoint
BASE_URL = "https://abq-transit-collector-test-548887462821.us-south1.run.app"

# Retriving Secret from google collab
API_KEY = userdata.get('YOUR_API_KEY')

# Create headers for authenticated requests
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

## Working API Endpoints:

- Service health check
  - BASE_URL/health
- Database statistics
  - BASE_URL/api/summary
- Recent vehicle data
  - BASE_URL/api/recent?hours=N
- List of all routes
  - BASE_URL/api/routes
- Specific route data
  - BASE_URL/api/route?name=X&hours=N
- Active vehicles
  - BASE_URL/api/vehicles
- Current vehicle positions for mapping
  - BASE_URL/api/locations

In [ ]:
# Test health endpoint (no auth required)
response = requests.get(f"{BASE_URL}/health")
print("Health Check:", response.json())

# Test authenticated endpoint
response = requests.get(f"{BASE_URL}/api/summary", headers=headers)
if response.status_code == 200:
    print("✅ API Authentication successful!")
    print("Summary:", response.json())
else:
    print("❌ Authentication failed:", response.status_code, response.text)

In [ ]:
def get_summary():
    """Get database summary statistics"""
    response = requests.get(f"{BASE_URL}/api/summary", headers=headers)
    return response.json()

summary = get_summary()
print(f"Total Records: {summary['total_records']:,}")
print(f"Unique Vehicles: {summary['unique_vehicles']}")
print(f"Unique Routes: {summary['unique_routes']}")
print(f"Collection Days: {summary['collection_days']}")
print(f"Latest Record: {summary['latest_record']}")

In [ ]:
summary

In [ ]:
def get_recent_data(hours=1):
    """Get recent vehicle data"""
    response = requests.get(
        f"{BASE_URL}/api/recent",
        params={"hours": hours},
        headers=headers
    )
    return response.json()

# Get last hour of data
recent = get_recent_data(hours=1)
print(f"Records in last hour: {recent['record_count']}")

# Convert to DataFrame
df_recent = pd.DataFrame(recent['data'])
df_recent['timestamp_collected'] = pd.to_datetime(df_recent['timestamp_collected'])
df_recent.head()

In [ ]:
def get_routes():
    """Get list of all routes"""
    response = requests.get(f"{BASE_URL}/api/routes", headers=headers)
    return response.json()

def get_route_data(route_name, hours=1):
    """Get data for specific route"""
    response = requests.get(
        f"{BASE_URL}/api/route",
        params={"name": route_name, "hours": hours},
        headers=headers
    )
    return response.json()

# Get all routes
routes = get_routes()
df_routes = pd.DataFrame(routes['routes'])
print(f"Total routes: {len(df_routes)}")
df_routes.head(10)

In [ ]:
def get_active_vehicles():
    """Get currently active vehicles"""
    response = requests.get(f"{BASE_URL}/api/vehicles", headers=headers)
    return response.json()

vehicles = get_active_vehicles()
print(f"Active vehicles: {vehicles['active_vehicles']}")
df_vehicles = pd.DataFrame(vehicles['vehicles'])
df_vehicles.head()